## 1. Import Python Libraries

In [1]:
import scipy
import pandas as pd
import numpy as np 
import seaborn as sns
from scipy.stats import qmc

## 2. Setup

- `FEATURE_NAMES` = Electrode Parameters
- `LB` = Lower Bounds
- `UB` = Upper Bounds
- `NUM_DIMENSIONS` = Number of Features (Features that are **NOT** calculated)
- `NUM_SAMPLES` = Number of Samples 
- `K` = Carman-Kozeny Constant

In [2]:
FEATURE_NAMES = ["ElectricalConductivity", "Porosity", "Avg_Fib_Diam", "Mean_Pore_Diam", "Permeability"]
LB = [60, 0.75, 1e-5, 1e-4]
UB = [110, 0.95, 2e-5, 2e-4]
NUM_DIMENSIONS = 4
NUM_SAMPLES = 200
K = 180
RANDOM_STATE = 23

## 3. Latin Hypercube Sampling `scipy.stats`

In [3]:
hypercube = qmc.LatinHypercube(d=NUM_DIMENSIONS, seed=RANDOM_STATE)
samples = hypercube.random(n=NUM_SAMPLES)
scaled_samples = qmc.scale(samples, LB, UB)

In [4]:
sample_plan = pd.DataFrame(scaled_samples, columns=FEATURE_NAMES[:-1])
sample_plan.head()

,ElectricalConductivity,Porosity,Avg_Fib_Diam,Mean_Pore_Diam
0,67.326517,0.926359,0.000014,0.000135
1,86.086664,0.818147,0.000020,0.000124
2,61.320854,0.873529,0.000018,0.000101
3,107.484037,0.769545,0.000017,0.000115
4,108.364926,0.870316,0.000017,0.000175


### Calculating Permeability 

In [5]:
sample_plan[FEATURE_NAMES[-1]] = (
    sample_plan["Avg_Fib_Diam"] ** 2
    * sample_plan["Porosity"] ** 3
    / (K * (1 - sample_plan["Porosity"]) ** 2)
)
        

In [6]:
# Printing the Final Sampling plan
sample_plan.head()

,ElectricalConductivity,Porosity,Avg_Fib_Diam,Mean_Pore_Diam,Permeability
0,67.326517,0.926359,0.000014,0.000135,1.698905e-10
1,86.086664,0.818147,0.000020,0.000124,3.548597e-11
2,61.320854,0.873529,0.000018,0.000101,7.651234e-11
3,107.484037,0.769545,0.000017,0.000115,1.359119e-11
4,108.364926,0.870316,0.000017,0.000175,6.123377e-11


In [7]:
sample_plan.to_csv("Sampling_Plan_LatinHypercube.csv")